In [42]:
import sys
import math
import pandas as pd
import spotipy
import spotipy.util as util

# Authorization for Spotify Profile Access

In [25]:
# Generate authorization token for reading user library
scope = 'user-library-read'
client_id = 'c3aacebd1b0d4a7ca415ca317ec72531'
client_secret = '343a5709a346489d9aa013f32f00cfe4'
redirect_uri = 'http://google.com/'

print("Please Enter Username")
username = str(input("Username: ")) #12124976904
print()

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)

else:
    print("Can't get token for", username)
    sys.exit()

Please Enter Username
Username: 12124976904



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=c3aacebd1b0d4a7ca415ca317ec72531&response_type=code&redirect_uri=http%3A%2F%2Fgoogle.com%2F&scope=user-library-read in your browser


Enter the URL you were redirected to: https://www.google.com/?code=AQDzbXtQQdO6t-f5RsqbtKUbft2ZhgQVTEDjmt12acxngWzBWzB0R_DraRGM5RUMvRfmNgxSl3SGLrr38TVpWqUmL7YYtJp7zN_hge_wlo4skA1y0ZSF2NhdDwZytTfiq7mijUXiC4dYmidTovT4N7dmro6v4xmwjiNM0swgFqpSrpip2_XxZ9VvsmUxtwVsuBQSI30pE7CwokI




# Data Collection and Tidying

In [48]:
def get_saved_tracks:
    result = sp.current_user_saved_tracks()
    tracks = result['items']

    while(result['next']):
        result = sp.next(result)
        tracks.extend(result['items'])

    return tracks

features = pd.DataFrame()
for track in tracks:
    features_dict = sp.audio_features(track['track']['uri'])[0]
    features = features.append(features_dict, ignore_index=True)
    
features = features[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'tempo', 'valence']]
features

,acousticness,danceability,energy,instrumentalness,liveness,tempo,valence
0,0.004860,0.410,0.6070,0.570000,0.1230,137.083,0.2370
1,0.000138,0.316,0.7220,0.753000,0.1250,119.642,0.2520
2,0.056500,0.442,0.5020,0.575000,0.1070,188.421,0.1670
3,0.000192,0.233,0.8300,0.732000,0.1400,136.880,0.4600
4,0.008380,0.350,0.9230,0.002320,0.0925,161.921,0.6180
5,0.035200,0.615,0.9310,0.000024,0.0914,137.544,0.5300
6,0.734000,0.625,0.1350,0.256000,0.1220,111.568,0.2310
7,0.525000,0.599,0.0639,0.870000,0.0888,62.319,0.1810
8,0.971000,0.142,0.0420,0.913000,0.0520,141.164,0.0427
9,0.890000,0.303,0.1290,0.007760,0.1230,84.167,0.1550


In [27]:
tracks

{'href': 'https://api.spotify.com/v1/me/tracks?offset=0&limit=20',
 'items': [{'added_at': '2019-12-05T02:22:23Z',
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4ABTeyDAbK3iv09CHSsoW0'},
       'href': 'https://api.spotify.com/v1/artists/4ABTeyDAbK3iv09CHSsoW0',
       'id': '4ABTeyDAbK3iv09CHSsoW0',
       'name': 'A Minor Forest',
       'type': 'artist',
       'uri': 'spotify:artist:4ABTeyDAbK3iv09CHSsoW0'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
      'JO',
  

{'album': {'album_type': 'ALBUM', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0iOk5xBBPcioC95IKkdWTd'}, 'href': 'https://api.spotify.com/v1/artists/0iOk5xBBPcioC95IKkdWTd', 'id': '0iOk5xBBPcioC95IKkdWTd', 'name': 'T.M.M.', 'type': 'artist', 'uri': 'spotify:artist:0iOk5xBBPcioC95IKkdWTd'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/7uTKhj0F6Qp7mrqgNXxI0r'}, 'href': 'https://api.spotify.com/v1/albums/7uTKhj0F6Qp7mrqgNXxI0r', 'id': '7uTKhj0F6Qp7mrqgNXx